In [1]:
from keras.models import Sequential, Model, load_model
from keras import applications
from keras import optimizers
from keras import metrics
from keras.layers import Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

img_rows, img_cols, img_channel = 224, 224, 3

base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, img_channel))

Using TensorFlow backend.


In [2]:
train_data_dir = 'data/train'
validation_data_dir = 'data/validate'
epochs = 16
batch_size = 16
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   rotation_range=30)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    seed = 1997)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    seed = 1997)


Found 13347 images belonging to 151 classes.
Found 4535 images belonging to 151 classes.


In [3]:
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(151, activation='softmax'))

In [4]:
for layer in base_model.layers[:15]:
    layer.trainable = False

In [5]:
model = Model(input= base_model.input, output= top_model(base_model.output))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`
  """Entry point for launching an IPython kernel.


In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [7]:
top3 = lambda x,y: metrics.top_k_categorical_accuracy(x,y,3)
top5 = lambda x,y: metrics.top_k_categorical_accuracy(x,y,5)
sgd = optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy',top3,top5])

In [8]:
model.fit_generator(
        train_generator,
        steps_per_epoch = 900,
        epochs = epochs,
        validation_data = validation_generator,
        validation_steps = 300)

Epoch 1/16
900/900 [==============================] - 823s - loss: 5.0242 - acc: 0.0074 - <lambda>_1: 0.0219 - <lambda>_2: 0.0369 - val_loss: 4.9991 - val_acc: 0.0134 - val_<lambda>_1: 0.0286 - val_<lambda>_2: 0.0428
Epoch 2/16
900/900 [==============================] - 772s - loss: 4.4707 - acc: 0.0944 - <lambda>_1: 0.1635 - <lambda>_2: 0.2076 - val_loss: 2.8436 - val_acc: 0.4156 - val_<lambda>_1: 0.5638 - val_<lambda>_2: 0.6312
Epoch 3/16
900/900 [==============================] - 772s - loss: 2.6826 - acc: 0.4070 - <lambda>_1: 0.5683 - <lambda>_2: 0.6328 - val_loss: 1.4863 - val_acc: 0.6788 - val_<lambda>_1: 0.7894 - val_<lambda>_2: 0.8328
Epoch 4/16
900/900 [==============================] - 772s - loss: 1.7470 - acc: 0.5910 - <lambda>_1: 0.7423 - <lambda>_2: 0.7977 - val_loss: 1.2046 - val_acc: 0.7153 - val_<lambda>_1: 0.8336 - val_<lambda>_2: 0.8687
Epoch 5/16
900/900 [==============================] - 773s - loss: 1.3269 - acc: 0.6765 - <lambda>_1: 0.8182 - <lambda>_2: 0.8626 - 

In [9]:
json_string = model.to_json()
model.save_weights('model_weights.h5')
with open("model.json", "w") as json_file:
    json_file.write(model_json)